In [93]:
# Credit: The Markov Chain Monte Carlo method to break the substitution cipher 
# is credited toward a class assignment at UC Berkeley
import numpy as np
import math
import matplotlib.pyplot as plt
from matplotlib import animation
import pandas as pd
import scipy.stats as stats
import utils

encrypted_message = "rajvzoocaomfvanzhs"

# quantify the accuracy as a function of message length
# improvements
# condition on previous 2 char







def metropolis_hastings(proposal_func, initial, acceptance_func,
                        num_iters, step=30):
    """
    proposal_func: function that proposes
        the next state given the current state as
        argument.
    init_func: function that proposes starting
        state; takes no arguments and returns a
        sample state
    score_func: function that calculates f(y)/f(x)
        * g(y,x)/g(x,y); takes in two state samples
        (the current sample x then the candidate y).
    
    Returns a sequence of sample states. The length of the final sequence is 
    num_iters // step.
    """
    samples = []
    sample = initial()
    for i in range(num_iters):
        proposed_state = proposal_func(sample)
        acceptance_prob = min(1, acceptance_func(sample, proposed_state))
        if np.random.uniform() < acceptance_prob:
            sample = proposed_state
        samples.append(sample)
    return samples[::step]

In [94]:
input_file = "Ulysses.txt"

def build_freq_matrix(input_file):
    """
    Builds a matrix that represents the transitional
    probabilities between letters in input_file.
    
    freq_matrix[i][j] is the probability of
    transitioning from the jth letter of the alphabet
    to the jst letter. ' ' (space) is denoted as the
    26th letter of the alphabet.
    """
    # create a 27 by 27 matrix of 1's. 
    counts = np.ones([27, 27])
    with open(input_file, 'r', encoding='utf8') as f:
        for _ in range(1000000):
            line = f.readline()
            if len(line) > 2:
                for i in range(len(line) - 2):
                    # if not an alphabetic character, then treat the charater as (space)
                    # ASCII capital letters start at 65.
                    first_char = ord(line[i].upper()) - 65 if line[i].isalpha() else 26
                    # same thing for the next character
                    second_char = ord(line[i+1].upper()) - 65 if line[i+1].isalpha() else 26
                    # <= 26 means ignore the characters that gives lower case letters because they could pass 
                    # the isalpha checkpoint.
                    if not (first_char == 26 and second_char == 26) and first_char <= 26 and second_char <= 26:
                        counts[first_char][second_char] += 1
        # tranposition is for dimension match
        transition_matrix = (counts.T / np.sum(counts, axis=1)).T
    return transition_matrix

# decrypt function takes an encrypted message and a key (ordering), gives you the corresponding plaintext (crypto term for the original message)
def decrypt(string, ordering):
    """
    ordering: a list representing the permutation
    """
    output_string = ""
    for i in string:
        # convert to capital letter and - 65 to get its place in the alphabet.
        char = ord(i.upper()) - 65 if i.isalpha() else 26
        # append to the output_string
        # if ordering[char] is 26, append a space, i.e. " "
        output_string += chr(ordering[char] + 65) if ordering[char] != 26 else " "
    return output_string

freq_matrix = build_freq_matrix(input_file)

In [95]:
# returns a random permutation of alphabets, as the starting state. We don't know a priori what the key is, 
# so we can't do anything speicial here
def starting_state():
    """
    Return a random permutation of indices representing the alphabet.
    """
    starting_ordering = list(range(0, 27))
    np.random.shuffle(starting_ordering)
    return starting_ordering

# sample the next key, might not make sense, but given enough trials, we'll arrive at the correct key. 
# theoretical question: how many iterations of algorithm does it take on average for us to converge to the right key?
def sample_next(sample):
    """
    To get a proposed cipher, swap 2 random letters
    """
    letters_to_swap = np.random.choice(27, 2)
    new_sample = sample[:]
    # swapping
    # if the swapping makes the the key less probable, we'll likely not accept. Massive rejections.
    new_sample[letters_to_swap[0]], new_sample[letters_to_swap[1]] = sample[letters_to_swap[1]], sample[letters_to_swap[0]]
    return new_sample

def make_log_f(encrypted_string, transition_matrix):
    """
    Generates a function which computes the 
    log of the function f in the description 
    (the probability of observing the sequence 
    of characters in the message decoded by 
    cipher x), which is then used to calculate 
    acceptance probabilities.
    
    encrypted_string: secret message string
    
    transition_matrix: for the simulated Markov process
    """
    def log_f(current_key):
        key_log_likelihood = 0
        # measure how likely a key is by adding up the probabilities of all the transitions
        for i in range(len(encrypted_string) - 1):
            first_char = ord(encrypted_string[i].upper()) - 65 if encrypted_string[i].isalpha() else 26
            second_char = ord(encrypted_string[i+1].upper()) - 65 if encrypted_string[i+1].isalpha() else 26
            # decrypt 
            decrypted_first, decrypted_second = current_key[first_char], current_key[second_char]
            key_log_likelihood += np.log(transition_matrix[decrypted_first][decrypted_second])
        return key_log_likelihood
    return log_f
    
def make_acceptance_func(encrypted_string, transition_matrix):
    """
    Calculate the acceptance probability, which is the quotient of 
    the probability of observing the message translated by the proposed key 
    devided by the probability of observing the message decrypted by the current
    key. 
    log is used for computational stability.
    
    encrypted_string: encrypted message string
    
    transition_matrix: for the simulated Markov process
    """
    def acceptance_func(current_key, proposed_key):
        sample_log_likelihood = log_f(current_key)
        proposed_log_likelihood = log_f(proposed_key)
        return np.exp(proposed_log_likelihood - sample_log_likelihood)
    return acceptance_func

log_f = make_log_f(encrypted_message, freq_matrix)
acceptance_func = make_acceptance_func(encrypted_message, freq_matrix)

In [96]:
samples = metropolis_hastings(sample_next, starting_state, acceptance_func, 4000, step=50)
for sample in samples[::len(samples) // 5]:
    print(decrypt(encrypted_message, sample), '\n')

MQLGICCNQCOSGQBITK 

U RDISSK SHED CIMO 

D ONESST SHAN BEWI 

P TYOLLD LABY COUN 

D ATOLLY LEST BOWI 



In [97]:
log_probs = [log_f(s) for s in samples]
best_cipher_index = np.argmax(log_probs)
print(decrypt(encrypted_message, samples[best_cipher_index]).upper())

D ATOLLY LEST COWI


In [98]:
# 4000 characters
encrypted_message2 = "Shbsibzbhjghtbgnsfwjizppcbzyqnmepwxuwxbhtzhbzbisnupwboznbsnblmiiwiismnbmvbzbummxbvmjhgnwbogihb wbsnbeznhbmvbzbesvwbTmewfwjbpshhpwbqnmenbhtwbvwwpsnuibmjbfsweibmvbigytbzboznbozcb wbmnbtsibvsjihbwnhwjsnubzbnwsut mgjtmmxbhtsibhjghtbsibimbewppbvsdwxbsnbhtwbosnxibmvbhtwbigjjmgnxsnubvzospswibhtzhbtwbsibymnisxwjwxbzibhtwbjsuthvgpbljmlwjhcbmvbimowbmnwbmjbmhtwjbmvbhtwsjbxzguthwjibOcbxwzjbOjb wnnwhbizsxbtsibpzxcbhmbtsobmnwbxzcbtzfwbcmgbtwzjxbhtzhbNwhtwjvswpxbLzjqbsibpwhbzhbpzihbOjb wnnwhbjwlpswxbhtzhbtwbtzxbnmhb ghbshbsibjwhgjnwxbitwbvmjbOjibPmnubtzibrgihb wwnbtwjwbznxbitwbhmpxbowbzppbz mghbshbOjb wnnwhbozxwbnmbzniewjbXmbcmgbnmhbeznhbhmbqnmebetmbtzibhzqwnbshbyjswxbtsibesvwbsolzhswnhpcbCmgbeznhbhmbhwppbowbznxbSbtzfwbnmbm rwyhsmnbhmbtwzjsnubshbHtsibezibsnfshzhsmnbwnmgutbEtcbocbxwzjbcmgbogihbqnmebOjibPmnubizcibhtzhbNwhtwjvswpxbsibhzqwnb cbzbcmgnuboznbmvbpzjuwbvmjhgnwbvjmobhtwbnmjhtbmvbWnupznxbhtzhbtwbyzowbxmenbmnbOmnxzcbsnbzbytzsiwbznxbvmgjbhmbiwwbhtwblpzywbznxbezibimbogytbxwpsuthwxbeshtbshbhtzhbtwbzujwwxbeshtbOjbOmjjsibsoowxszhwpcbhtzhbtwbsibhmbhzqwblmiiwiismnb wvmjwbOsytzwpozibznxbimowbmvbtsibiwjfznhibzjwbhmb wbsnbhtwbtmgiwb cbhtwbwnxbmvbnwdhbewwqbEtzhbsibtsibnzowb snupwcbSibtwbozjjswxbmjbisnupwbMtbisnupwbocbxwzjbhmb wbigjwbZbisnupwboznbmvbpzjuwbvmjhgnwbvmgjbmjbvsfwbhtmgiznxbzbcwzjbEtzhbzbvsnwbhtsnubvmjbmgjbusjpibTmebimbtmebyznbshbzvvwyhbhtwobOcbxwzjbOjb wnnwhbjwlpswxbtsibesvwbtmebyznbcmgb wbimbhsjwimowbCmgbogihbqnmebhtzhbSbzobhtsnqsnubmvbtsibozjjcsnubmnwbmvbhtwobSibhtzhbtsibxwisunbsnbiwhhpsnubtwjwbXwisunbnmniwniwbtmebyznbcmgbhzpqbimb ghbshbsibfwjcbpsqwpcbhtzhbtwbozcbvzppbsnbpmfwbeshtbmnwbmvbhtwobznxbhtwjwvmjwbcmgbogihbfsishbtsobzibimmnbzibtwbymowibSbiwwbnmbmyyzismnbvmjbhtzhbCmgbznxbhtwbusjpibozcbumbmjbcmgbozcbiwnxbhtwob cbhtwoiwpfwibetsytblwjtzlibesppb wbihsppb whhwjbvmjbzibcmgbzjwbzibtznximowbzibzncbmvbhtwobOjb snupwcbosuthbpsqwbcmgbhtwb wihbmvbhtwblzjhcbOcbxwzjbcmgbvpzhhwjbowbSbywjhzsnpcbtzfwbtzxbocbitzjwbmvb wzghcb ghbSbxmbnmhbljwhwnxbhmb wbznchtsnubwdhjzmjxsnzjcbnmebEtwnbzbemoznbtzibvsfwbujmenbglbxzguthwjibitwbmguthbhmbusfwbmfwjbhtsnqsnubmvbtwjbmenb wzghcbSnbigytbyziwibzbemoznbtzibnmhbmvhwnbogytb wzghcbhmbhtsnqbmvb ghbocbxwzjbcmgbogihbsnxwwxbumbznxbiwwbOjb snupwcbetwnbtwbymowibsnhmbhtwbnwsut mgjtmmxbShbsibomjwbhtznbSbwnuzuwbvmjbSbziigjwbcmgb ghbymnisxwjbcmgjbxzguthwjibMnpcbhtsnqbetzhbznbwihz psitownhbshbemgpxb wbvmjbmnwbmvbhtwobIsjbEsppszobznxbPzxcbPgyzibzjwbxwhwjosnwxbhmbumbowjwpcbmnbhtzhbzyymgnhbvmjbsnbuwnwjzpbcmgbqnmebhtwcbfsishbnmbnweymowjibSnxwwxbcmgbogihbumbvmjbshbesppb wbsolmiis pwbvmjbgibhmbfsishbtsobsvbcmgbxmbnmhbCmgbzjwbmfwj-iyjglgpmgibigjwpcbSbxzjwbizcbOjb snupwcbesppb wbfwjcbupzxbhmbiwwbcmgbznxbSbesppbiwnxbzbvwebpsnwib cbcmgbhmbziigjwbtsobmvbocbtwzjhcbymniwnhbhmbtsibozjjcsnubetsytwfwjbtwbytmmiwibmvbhtwbusjpibhtmgutbSbogihbhtjmebsnbzbummxbemjxbvmjbocbpshhpwbPsaacbSbxwisjwbcmgbesppbxmbnmbigytbhtsnubPsaacbsibnmhbzb shb whhwjbhtznbhtwbmhtwjibznxbSbzobigjwbitwbsibnmhbtzpvbimbtznximowbzibRznwbnmjbtzpvbimbummx-tgomgjwxbzibPcxszb ghbcmgbzjwbzpezcibusfsnubtwjbhtwbljwvwjwnywbHtwcbtzfwbnmnwbmvbhtwobogytbhmbjwymoownxbhtwobjwlpswxbtwbhtwcbzjwbzppbisppcbznxbsunmjznhbpsqwbmhtwjbusjpib ghbPsaacbtzibimowhtsnubomjwbmvbkgsyqnwiibhtznbtwjbisihwjibOjb wnnwhbtmebyznbcmgbz giwbcmgjbmenbytspxjwnbsnbigytbzbezcbCmgbhzqwbxwpsuthbsnbfwdsnubowbCmgbtzfwbnmbymolziismnbvmjbocblmmjbnwjfwibCmgbosihzqwbowbocbxwzjbSbtzfwbzbtsutbjwilwyhbvmjbcmgjbnwjfwibHtwcbzjwbocbmpxbvjswnxibSbtzfwbtwzjxbcmgbownhsmnbhtwobeshtbymnisxwjzhsmnbhtwiwbpzihbhewnhcbcwzjibzhbpwzihbZtbcmgbxmbnmhbqnmebetzhbSbigvvwjb ghbSbtmlwbcmgbesppbuwhbmfwjbshbznxbpsfwbhmbiwwbozncbcmgnubownbmvbvmgjbhtmgiznxbzbcwzjbymowbsnhmbhtwbnwsut mgjtmmxbShbesppb wbnmbgiwbhmbgibsvbhewnhcbigytbitmgpxbymowbisnywbcmgbesppbnmhbfsishbhtwobXwlwnxbglmnbshbocbxwzjbhtzhbetwnbhtwjwbzjwbhewnhcbSbesppbfsishbhtwobzppbOjb wnnwhbezibimbmxxbzbosdhgjwbmvbkgsyqblzjhibizjyzihsybtgomgjbjwiwjfwbznxbyzljsywbhtzhbhtwbwdlwjswnywbmvbhtjww-znx-hewnhcbcwzjibtzxb wwnbsnigvvsyswnhbhmbozqwbtsibesvwbgnxwjihznxbtsibytzjzyhwjbTwjbosnxbezibpwiibxsvvsygphbhmbxwfwpmlbItwbezibzbemoznbmvbowznbgnxwjihznxsnubpshhpwbsnvmjozhsmnbznxbgnywjhzsnbhwolwjbEtwnbitwbezibxsiymnhwnhwxbitwbvznyswxbtwjiwpvbnwjfmgibHtwb gisnwiibmvbtwjbpsvwbezibhmbuwhbtwjbxzguthwjibozjjswxbshibimpzywbezibfsishsnubznxbnwei"
# 8000 characters
encrypted_message3 = "Rm z ivnlgv erooztv mvhgovw rm gsv iloormt srooh lu z ozmw uzi zdzb uiln gsv yfhgormt xrgrvh zmw gldmh gsviv orevw z xlnnfmrgb lu kvlkov dsl ovw hrnkov bvg xlmgvmg orevh Gsv erooztvih ilhv drgs gsv hfm zmw dlipvw grivovhhob gsilfts gsv wzb gvmwrmt gl gsvri urvowh zmw orevhglxp yvuliv ivgrirmt zg wfhp gl gsv dzings zmw xlnulig lu gsvri slnvh Vzxs wzb dzh z szinlmrlfh yovmw lu sziw dlip zmw xznzizwvirv drgs gsv erooztvih lugvm tzgsvirmt rm gsv vevmrmth zilfmw z ozitv xlnnfmzo uriv gl hsziv hglirvh hrmt hlmth zmw xvovyizgv gsv hnzoo qlbh lu oruv Gsrh erooztv dzh fmorpv zmb lgsvi rg dzh z kozxv dsviv grnv hvvnvw gl hgzmw hgroo dsviv gizwrgrlmh dviv fksvow drgs kirwv zmw dsviv gsv ylmwh yvgdvvm kvlkov dviv hgilmt zmw fmyivzpzyov Gsv erooztv dzh hfiilfmwvw yb ofhs tivvm urvowh gszg hgivgxsvw zh uzi zh gsv vbv xlfow hvv rmgviifkgvw lmob yb gsv lxxzhrlmzo xofnk lu givvh li drmwrmt hgivzn Gsvhv urvowh dviv gsv oruvyollw lu gsv erooztv kilerwrmt gsv ullw zmw ivhlfixvh mvvwvw uli gsv erooztvih gl hfhgzrm gsvnhvoevh Gsilftslfg gsv bvzi gsv erooztvih dlipvw gltvgsvi gl kozmg gvmw zmw szievhg xilkh gsvri vuuligh vmhfirmt gszg ml lmv vevi dvmg sfmtib Gsv xszmtv lu hvzhlmh yilftsg mvd xszoovmtvh zmw lkkligfmrgrvh uiln gsv kozmgrmt lu hvvwh rm gsv hkirmt gl gsv szievhg rm gsv uzoo vzxs hvzhlm dzh nzipvw yb rgh ldm fmrjfv gzhph zmw irgfzoh Rm gsv xvmgvi lu gsv erooztv hgllw z ozitv lzp givv rgh zmxrvmg yizmxsvh hkivzwrmt drwv gl luuvi hszwv zmw hsvogvi yvmvzgs rgh ylftsh Gsv lzp givv dzh z kozxv lu tzgsvirmt zmw ivuovxgrlm dsviv erooztvih dlfow xlnv gl ivhg zugvi z olmt wzb lu dlip li gl hvvp hlozxv rm grnvh lu gilfyov Rgh kivhvmxv dzh z xlnuligrmt ivnrmwvi lu gsv xlmgrmfrgb lu oruv zmw gsv vmwfirmt hgivmtgs lu mzgfiv Mvzi gsv lzp givv dzh gsv erooztv hjfziv z yfhgormt sfy lu zxgrergb dsviv erooztvih dlfow xlnv gl gizwv tllwh vcxszmtv mvdh zmw hlxrzorav drgs gsvri mvrtsylih Gsv hjfziv dzh lugvm uroovw drgs gsv hlfmwh lu ozftsgvi zmw xlmevihzgrlm zh xsrowivm kozbvw zmw zwfogh vmtztvw rm zmrnzgvw wrhxfhhrlmh zylfg gsv vevmgh lu gsv wzb Rg dzh z kozxv lu xlmmvxgrlm dsviv gsv grvh gszg ylfmw gsv xlnnfmrgb gltvgsvi dviv hgivmtgsvmvw zmw ivrmulixvw Gsv erooztvih dviv z wrevihv tilfk vzxs drgs gsvri ldm fmrjfv hprooh zmw gzovmgh gszg gsvb yilftsg gl gsv xloovxgrev vuulig Gsviv dzh gsv yozxphnrgs dsl glrovw yb gsv ulitv hszkrmt nvgzo rmgl glloh zmw rnkovnvmgh gszg dviv vhhvmgrzo uli wzrob oruv Gsv yzpvi ilhv yvuliv wzdm gl kivkziv gsv yivzw zmw kzhgirvh gszg uroovw gsv zri drgs gsvri vmgrxrmt zilnz Gsv dvzevi hkfm gsivzwh rmgl xolgs xivzgrmt tzinvmgh zmw yozmpvgh gszg kilerwvw dzings zmw xlnulig wfirmt gsv xlow drmgvi nlmgsh Gsviv dviv zohl gsv uzinvih dsl ozylivw rm gsv urvowh kozmgrmt gvmwrmt zmw szievhgrmt gsv xilkh gszg hfhgzrmvw gsv erooztv Gsvhv nvm zmw dlnvm pmvd gsv ozmw rmgrnzgvob fmwvihgzmwrmt rgh isbgsnh zmw xbxovh zmw dliprmt rm szinlmb drgs mzgfiv gl xlzc uligs rgh ylfmgb Gsv hsvksviwh gvmwvw gl gsvri uolxph tfrwrmt gsvn gl ofhs kzhgfivh zmw kilgvxgrmt gsvn uiln kivwzglih Gsvri hproo zmw ertrozmxv vmhfivw gszg gsv erooztv szw z hgvzwb hfkkob lu dllo zmw nvzg Gsv erooztv svzovi z drhv zmw pmldovwtvzyov dlnzm gvmwvw gl gsv hrxp zmw rmqfivw fhrmt svi pmldovwtv lu sviyh zmw ivnvwrvh gl xfiv zronvmgh zmw vzhv kzrm Hsv dzh z urtfiv lu ivhkvxg zmw ivevivmxv svi kivhvmxv kilerwrmt xlnulig zmw ivzhhfizmxv gl gslhv rm mvvw Vzxs nvnyvi lu gsv xlnnfmrgb kozbvw z ergzo ilov xlmgiryfgrmt gsvri vuuligh zmw vckvigrhv gl gsv xloovxgrev tllw Gsv erooztvih fmwvihgllw gszg gsvri hgivmtgs ozb rm gsvri fmrgb zmw xllkvizgrlm zmw gsvb dlipvw gltvgsvi gl levixlnv gsv xszoovmtvh gszg oruv kivhvmgvw Gsilftslfg gsv bvzi gsv erooztv xvovyizgvw z mfnyvi lu uvhgrezoh zmw vevmgh gszg yilftsg gsv xlnnfmrgb gltvgsvi rm qlblfh xvovyizgrlm Gsvhv lxxzhrlmh dviv nzipvw yb uvzhgrmt wzmxrmt zmw nfhrx zh gsv erooztvih gllp z yivzp uiln gsvri ozylih gl vmqlb vzxs lgsvi'h xlnkzmb zmw trev gszmph uli gsvri yovhhrmth Gsv szievhg uvhgrezo dzh lmv lu gsv nlhg vztviob zmgrxrkzgvw vevmgh lu gsv bvzi z grnv dsvm gsv erooztvih tzgsvivw gl xvovyizgv gsv uifrgh lu gsvri ozyli Gsv erooztv hjfziv dzh wvxlizgvw drgs xloliufo yzmmvih zmw tziozmwh zmw gzyovh dviv ozwvm drgs zm zyfmwzmxv lu ullw zmw wirmp Gsv zri dzh uroovw drgs gsv hlfmwh lu nfhrx zmw ozftsgvi zh gsv erooztvih wzmxvw zmw hzmt ozgv rmgl gsv mrtsg Gsv uvhgrezo dzh z grnv lu gszmphtrermt z nlnvmg gl ivuovxg lm gsv sziw dlip zmw wvwrxzgrlm gszg szw yilftsg gsv szievhg gl uifrgrlm zmw gl vckivhh tizgrgfwv uli gsv ylfmgrufo trugh lu gsv ozmw Zmlgsvi rnkligzmg vevmg dzh gsv hkirmt kozmgrmt uvhgrezo z grnv dsvm gsv erooztvih xznv gltvgsvi gl hld gsv hvvwh gszg dlfow tild rmgl gsv xilkh gszg hfhgzrmvw gsvn Gsrh uvhgrezo dzh nzipvw yb z hvmhv lu slkv zmw ivmvdzo zh gsv erooztvih ollpvw ulidziw gl gsv kilnrhv lu z mvd tildrmt hvzhlm Gsvb dlipvw hrwv yb hrwv rm gsv urvowh gsvri vuuligh z gvhgznvmg gl gsvri hszivw xlnnrgnvmg gl gsv dvoo-yvrmt lu gsv xlnnfmrgb Gsv erooztv dzh zohl slnv gl z mfnyvi lu gizwrgrlmh zmw xfhglnh gszg szw yvvm kzhhvw wldm gsilfts tvmvizgrlmh Gsvhv gizwrgrlmh dviv zm rmgvtizo kzig lu gsv erooztvih' orevh kilerwrmt z hvmhv lu xlmgrmfrgb zmw xlmmvxgrlm gl gsvri zmxvhglih Lmv hfxs gizwrgrlm dzh gsv zmmfzo hglibgvoormt mrtsg z grnv dsvm erooztvih tzgsvivw zilfmw gsv xlnnfmzo uriv gl hsziv gzovh lu svilrhn zwevmgfiv zmw drhwln Gsvhv hglirvh dviv nliv gszm qfhg vmgvigzrmnvmg gsvb dviv z dzb lu kivhviermt gsv srhglib zmw ezofvh lu gsv xlnnfmrgb zmw lu rnkzigrmt rnkligzmg ovhhlmh gl gsv blfmtvi tvmvizgrlm Zmlgsvi xsvirhsvw gizwrgrlm dzh gsv kizxgrxv lu szmwxizugrmt trugh uli olevw lmvh wfirmt gsv drmgvi nlmgsh Zh gsv wzbh tivd hsligvi zmw gsv mrtsgh xlowvi erooztvih dlfow tzgsvi rm gsvri slnvh gl xivzgv yvzfgrufo zmw fmrjfv rgvnh gszg dlfow yv vcxszmtvw wfirmt gsv drmgvi hlohgrxv xvovyizgrlm Gsrh gizwrgrlm dzh z dzb lu vckivhhrmt olev zmw zkkivxrzgrlm zmw lu hgivmtgsvmrmt gsv ylmwh yvgdvvm uznrob zmw uirvmwh Gsv erooztv dzh z kozxv dsviv veviblmv dzh ezofvw zmw ivhkvxgvw ivtziwovhh lu gsvri ztv li zyrorgrvh Gsv vowvih dviv svow rm srts vhgvvn gsvri drhwln zmw vckvirvmxv tfrwrmt gsv xlnnfmrgb zmw kilerwrmt z ormp gl gsv kzhg Xsrowivm dviv xsvirhsvw zmw mfigfivw vmxlfiztvw gl ovzim zmw tild rm z hfkkligrev zmw olermt vmerilmnvmg Gsv erooztvih yvorvevw rm gsv rnkligzmxv lu vwfxzgrlm zmw nzwv hfiv gszg vevib xsrow szw gsv lkkligfmrgb gl ovzim gl ivzw dirgv zmw zxjfriv gsv hprooh gsvb dlfow mvvw gl xlmgiryfgv gl gsv xlnnfmrgb Rm gsv vevmrmth zugvi gsv dlip dzh wlmv uznrorvh dlfow tzgsvi zilfmw gsv svzigs gl hsziv hglirvh zmw gvzxs gsv xsrowivm zylfg gsv dliow zilfmw gsvn Gsvhv nlnvmgh lu gltvgsvimvhh dviv xsvirhsvw zh gsvb ivrmulixvw gsv ezofvh zmw gizwrgrlmh gszg dviv zg gsv svzig lu gsv xlnnfmrgb Zh gsv bvzih kzhhvw gsv erooztv xlmgrmfvw gl gsirev z gvhgznvmg gl gsv hgivmtgs zmw ivhrorvmxv lu rgh kvlkov Wvhkrgv gsv xszoovmtvh zmw sziwhsrkh gsvb uzxvw gsv erooztvih ivnzrmvw hgvzwuzhg rm gsvri xlnnrgnvmg gl vzxs lgsvi zmw gl gsv ozmw gszg hfhgzrmvw gsvn Gsvb fmwvihgllw gszg gsvri hfxxvhh dzh mlg nvzhfivw yb dvzogs li nzgvirzo klhhvhhrlmh yfg yb gsv hgivmtgs lu gsvri xlnnfmrgb zmw gsv jfzorgb lu gsvri ivozgrlmhsrkh drgs vzxs lgsvi zmw drgs gsv mzgfizo dliow zilfmw gsvn Gsrh erooztv gslfts ivnlgv zmw rhlozgvw dzh z hsrmrmt vcznkov lu dszg xlfow yv zxsrvevw dsvm kvlkov dlipvw gltvgsvi rm szinlmb drgs z hszivw hvmhv lu kfiklhv zmw z wvvk ivhkvxg uli vzxs lgsvi zmw gsv ozmw gsvb xzoovw slnv"
# 16000 characters
encrypted_message4 = "R ahsvasvzjhamuazafvjwz hafzoovca vihovwayvhevv ajmoor tasrooiaz waigjjmg wvwaycawv ivaumjvihiahsvjvaozcazafrooztvag orpvaz camhsvjaHsriafrooztvaeziazalozxvaesvjvahsvajschsnamuaoruvaeziawrxhzhvwaycahsvaivzim iaz waesvjvahsvalvmlovaorfvwar aszjnm caerhsa zhgjvaHsvafrooztvjiaevjvazaxomivap rhaxmnng rhcaymg waycahjzwrhrm iaz wazaiszjvwasrihmjcahszhaihjvhxsvwayzxpahsjmgtsahsvatv vjzhrm iaVzxsawzcar ahsvafrooztvayvtz aerhsahsvaurjihaortshamuawze aziahsvaig 'iajzcialrvjxvwahsjmgtsahsvaxz mlcamuahjvviaz wayzhsvwahsvafzoovcar azatmowv atomeaHsvafrooztvjiaemgowajrivaujmnahsvrjaiognyvjaz waivhazymghahsvrjawzrocahzipiaerhsazaiv ivamualgjlmivaz wawvhvjnr zhrm aHsvauzjnvjiaemgowasvzwahmahsvrjaurvowiahmahv wahmahsvaxjmliahszhaljmfrwvwaigihv z xvaumjahsvaxmnng rhcaesrovahsvaisvlsvjwiatgrwvwahsvrjauomxpiahmahsvaogisalzihgjviahszhaor vwahsvasrooirwviaHsvayozxpinrhs'iaumjtvaemgowaxmnvahmaoruvaerhsahsvajr tr tamuasznnvjam az froaziasvaiszlvwanvhzoar hmahmmoiaz warnlovnv hiaviiv hrzoaumjahsvafrooztvji'aemjpaHsvayzpvjaemgowajrivavfv avzjorvjahsz ahsvamhsvjiahmaljvlzjvahsvawmgtsaumjahsvayjvzwaz walzihjrviahszhauroovwahsvazrjaerhsahsvrjahz hzorbr tazjmnzaYcahsvahrnvahsvaig aszwaugoocajriv ahsvafrooztvaikgzjvaemgowayvazaygihor tasgyamuazxhrfrhcazialvmlovatzhsvjvwahmahjzwvatmmwiavdxsz tva veiaz waiszjvar ahsvaxznzjzwvjrvahszhawvur vwahsvrjaezcamuaoruvaHsvafrooztvaeziaigjjmg wvwaycazalzhxsemjpamuaurvowiaz wanvzwmeiavzxsam vazahvihznv hahmahsvaszjwaemjpaz wawvwrxzhrm amuahsvauzjnvjiaesmahv wvwahsvnaHsvivaurvowiaevjvahsvaoruvyommwamuahsvafrooztvaljmfrwr tahsvatjzr iafvtvhzyoviaz waujgrhiahszhaigihzr vwahsvafrooztvjiahsjmgtsmghahsvacvzjaR ahsvailjr tahsvaurvowiaemgowaxmnvazorfvaerhsahsvafryjz hatjvv amua veatjmehsaziaivvwiaime ar ahsvauvjhrovaimroayvtz ahmailjmghaz wauomgjrisaHsvauzjnvjiaemjpvwahrjvoviiocahma gjhgjvahsvivacmg taloz hiahsvrjavuumjhiav igjr tazaymg hrugoaszjfvihar ahsvanm hsiahmaxmnvaZiaignnvjazjjrfvwahsvaurvowiaemgowayvahjz iumjnvwar hmazaivzamuatmowaziahsvaxjmliajrlv vwag wvjahsvaezjnaig aHsvafrooztvjiaemgowaxmnvahmtvhsvjahmajvzlahsvaszjfvihaemjpr tairwvaycairwvahmatzhsvjahsvaujgrhiamuahsvrjaozymjaHsvazrjaemgowayvauroovwaerhsahsvaimg wiamuaozgtshvjaz waim taziahsvafrooztvjiaxvovyjzhvwahsvaxgonr zhrm amuahsvrjaszjwaemjpaHsvaszjfvihauvihrfzoaeziam vamuahsvanmihavztvjocaz hrxrlzhvwavfv hiamuahsvacvzjazahrnvamuaqmcaz wahsz pitrfr tanzjpvwaycauvzihr tawz xr taz wangirxaHsvafrooztvaikgzjvaemgowayvazwmj vwaerhsaxmomjugoayz  vjiaz watzjoz wiaz wahzyoviaemgowayvaozwv aerhsaz azyg wz xvamuaummwaz wawjr paHsvafrooztvjiaemgowawz xvaozhvar hmahsva rtshahsvrjailrjrhiaoruhvwaycahsvaiv ivamuaxmnng rhcaz wahsvaizhriuzxhrm amuazaqmyaevooawm vaZiahsvawzciatjveaismjhvjaz wahsvazrjahgj vwaxjrilaerhsahsvazjjrfzoamuazghgn ahsvafrooztvjiaemgowaljvlzjvaumjahsvaxmowvjanm hsiazsvzwaHsvcaemgowatzhsvjaemmwaumjahsvrjaurjvialjvivjfvaummwaumjahsvaer hvjaz wav igjvahszhahsvrjasmnviaevjvajvzwcahmaerhsihz wahsvaxsrooamuaer hvjaHsvaurvowiaemgowayvalomevwaz waljvlzjvwaumjahsva vdhaloz hr taivzim ahsvaimroahgj vwaz wav jrxsvwaerhsahsvajvn z hiamuahsvaszjfvihaHsvafrooztvaemgowahzpvam azakgrvhvjanmjvajvuovxhrfvanmmwaziahsvafrooztvjiahgj vwar ezjwaljvlzjr taumjahsvaom taer hvjanm hsiaEr hvjar ahsvafrooztvaeziazahrnvamuajvihaz wajv vezoazaxsz xvaumjahsvafrooztvjiahmajvxszjtvahsvrjailrjrhiaz wajvxm  vxhaerhsahsvrjauznrorviaHsvaurvowiaozcawmjnz hag wvjazayoz pvhamuai meahsvahjvviayzjvaz wairov har ahsvaxmowaHsvafrooztvjiaemgowatzhsvjazjmg wahsvrjasvzjhsiaiszjr taihmjrviaz waim tiaziahsvcaezrhvwaumjahsvajvhgj amuailjr taHsvaer hvjaimoihrxvaeziazalzjhrxgozjocailvxrzoahrnvanzjpvwaycahsvavdxsz tvamuasz wnzwvatruhiaz wahsvaxvovyjzhrm amuahsvahgj r tamuahsvacvzjaRhaeziazahrnvamuasmlvaz wajv vezoazajvnr wvjahszhavfv ar ahsvawzjpvihawzciahsvaortshaemgowajvhgj az waoruvaemgowayvtr az veaHsjmgtsmghahsvacvzjahsvafrooztvaeziatgrwvwaycazaivhamuahjzwrhrm iaz waxgihmniahszhaszwayvv alziivwawme ahsjmgtsahsvatv vjzhrm iaHsvivahjzwrhrm iaevjvahsvasvzjhamuahsvaxmnng rhcaljmfrwr tazaiv ivamuaxm hr grhcaz waxm  vxhrm ahmahsvalzihaM vaigxsahjzwrhrm aeziahsvaihmjchvoor ta rtshazahrnvaesv afrooztvjiaemgowatzhsvjazjmg wazaozjtvaxmnng zoaurjvahmaiszjvahzoviamuasvjmrinazwfv hgjvaz waeriwmnaHsvivaihmjrviaevjvanmjvahsz aqgihav hvjhzr nv hahsvcaevjvazaezcamualjvivjfr tahsvasrihmjcaz wafzogviamuahsvaxmnng rhcaz wamuarnlzjhr tarnlmjhz haoviim iahmahsvacmg tvjatv vjzhrm aZ mhsvjaxsvjrisvwahjzwrhrm aeziahsvaxjzuhr tamuatruhiawgjr tahsvaer hvjanm hsiaZiahsvawzciatjveaismjhvjaz wahsva rtshiaxmowvjafrooztvjiaemgowatzhsvjar ahsvrjasmnviahmaxjvzhvayvzghrugoaz wag rkgvarhvniahszhaemgowayvavdxsz tvwawgjr tahsvaer hvjaimoihrxvaxvovyjzhrm aHsriahjzwrhrm aeziazaezcamuavdljviir taomfvaz wazlljvxrzhrm az wamuaihjv thsv r tahsvaym wiayvhevv auznrocaz waujrv wiaHsvafrooztvjiaevjvazawrfvjivatjmglavzxsaerhsahsvrjame ag rkgvaiprooiaz wahzov hiahszhahsvcayjmgtshahmahsvaxmoovxhrfvavuumjhaHsvjvaeziahsvayozxpinrhsaesmahmrovwaycahsvaumjtvaiszlr tanvhzoar hmahmmoiaz warnlovnv hiahszhaevjvaviiv hrzoaumjawzrocaoruvaHsvayzpvjajmivayvumjvawze ahmaljvlzjvahsvayjvzwaz walzihjrviahszhauroovwahsvazrjaerhsahsvrjav hrxr tazjmnzaHsvaevzfvjailg ahsjvzwiar hmaxomhsaxjvzhr tatzjnv hiaz wayoz pvhiahszhaljmfrwvwaezjnhsaz waxmnumjhawgjr tahsvaxmowaer hvjanm hsiaHsvauzjnvjiaesmaozymjvwar ahsvaurvowialoz hr tahv wr taz waszjfvihr tahsvaxjmliahszhaigihzr vwahsvafrooztvaevjvanv az waemnv aesmap veahsvaoz war hrnzhvocag wvjihz wr tarhiajschsniaz waxcxoviaz waemjpr tar aszjnm caerhsa zhgjvahmaxmzdaumjhsarhiaymg hcaHsvaisvlsvjwiahv wvwahmahsvrjauomxpiatgrwr tahsvnahmaogisalzihgjviaz waljmhvxhr tahsvnaujmnaljvwzhmjiaHsvrjaiprooaz wafrtroz xvav igjvwahszhahsvafrooztvaszwazaihvzwcaigllocamuaemmoaz wanvzhaHsvafrooztvasvzovjazaerivaz wap meovwtvzyovaemnz ahv wvwahmahsvairxpaz war qgjvwagir tasvjap meovwtvamuasvjyiaz wajvnvwrviahmaxgjvazronv hiaz wavzivalzr aIsvaeziazaurtgjvamuajvilvxhaz wajvfvjv xvasvjaljviv xvaljmfrwr taxmnumjhaz wajvziigjz xvahmahsmivar a vvwaVzxsanvnyvjamuahsvaxmnng rhcalozcvwazafrhzoajmovaxm hjryghr tahsvrjavuumjhiaz wavdlvjhrivahmahsvaxmoovxhrfvatmmwaHsvafrooztvjiag wvjihmmwahszhahsvrjaihjv thsaozcar ahsvrjag rhcaz waxmmlvjzhrm az wahsvcaemjpvwahmtvhsvjahmamfvjxmnvahsvaxszoov tviahszhaoruvaljviv hvwaWvilrhvahsvaszjwisrliahsvcauzxvwahsvafrooztvjiajvnzr vwaihvzwuzihar ahsvrjaxmnnrhnv hahmavzxsamhsvjaz wahmahsvaoz wahszhaigihzr vwahsvnaHsvcag wvjihmmwahszhahsvrjaigxxviiaezia mhanvzigjvwaycaevzohsamjanzhvjrzoalmiiviirm iayghaycahsvaihjv thsamuahsvrjaxmnng rhcaz wahsvakgzorhcamuahsvrjajvozhrm isrliaerhsavzxsamhsvjaz waerhsahsva zhgjzoaemjowazjmg wahsvnaHsriafrooztvahsmgtsajvnmhvaz warimozhvwaeziazaisr r tavdznlovamuaeszhaxmgowayvazxsrvfvwaesv alvmlovaemjpvwahmtvhsvjar aszjnm caerhsazaiszjvwaiv ivamualgjlmivaz wazawvvlajvilvxhaumjavzxsamhsvjaz wahsvaoz wahsvcaxzoovwasmnvaHsvaivzim iaxm hr gvwahmahgj az wahsvafrooztvahsjrfvwarhialvmlovaorfr tar aszjnm caerhsahsvaoz waz waerhsavzxsamhsvjaHsvcaxvovyjzhvwahsvanrovihm viamuaoruvahmtvhsvjaujmnayrjhsiaz waevwwr tiahmahsvalziir tamuaomfvwam viavzxsavfv hazahvihznv hahmahsvav wgjr taihjv thsamuahsvrjaxmnng rhcaHsvafrooztvaeziazalozxvamuaozgtshvjaz waim tamuaszjwaemjpaz waxvovyjzhrm amuaomfvaz waomiiaHsjmgtsarhazooahsvafrooztvjiajvnzr vwaymg waycahsvahjzwrhrm iaz wafzogviahszhaszwayvv alziivwawme ahsjmgtsahsvatv vjzhrm iahsvrjaorfviazahvihznv hahmahsvalmevjamuaxmnng rhcaz wahsvaihjv thsamuahsvasgnz ailrjrhaZiahsvacvzjialziivwahsvafrooztvaxm hr gvwahmahsjrfvazahvihznv hahmahsvajvirorv xvaz wawvhvjnr zhrm amuarhialvmlovaWvilrhvahsvaxszoov tviaz waszjwisrliahsvcauzxvwahsvafrooztvjiajvnzr vwaihvzwuzihar ahsvrjaxmnnrhnv hahmavzxsamhsvjaz wahmahsvaoz wahszhaigihzr vwahsvnaHsvcag wvjihmmwahszhahsvrjaigxxviiaezia mhanvzigjvwaycaevzohsamjanzhvjrzoalmiiviirm iayghaycahsvaihjv thsamuahsvrjaxmnng rhcaz wahsvakgzorhcamuahsvrjajvozhrm isrliaerhsavzxsamhsvjaz waerhsahsva zhgjzoaemjowazjmg wahsvnaHsriafrooztvahsmgtsajvnmhvaz warimozhvwaeziazaisr r tavdznlovamuaeszhaxmgowayvazxsrvfvwaesv alvmlovaemjpvwahmtvhsvjar aszjnm caerhsazaiszjvwaiv ivamualgjlmivaz wazawvvlajvilvxhaumjavzxsamhsvjaz wahsvaoz wahsvcaxzoovwasmnvaHsriafrooztvahsmgtsajvnmhvaz warimozhvwaeziazaisr r tavdznlovamuaeszhaxmgowayvazxsrvfvwaesv alvmlovaemjpvwahmtvhsvjar aszjnm caerhsazaiszjvwaiv ivamualgjlmivaz wazawvvlajvilvxhaumjavzxsamhsvjaz wahsvaoz wahsvcaxzoovwasmnvaR ahsvasvzjhamuahsriafvjwz hafzoovcahsvjvaozcazafrooztvag orpvaz camhsvjazalozxvaesvjvahsvajschsnamuaoruvaeziawrxhzhvwaycahsvaivzim iaz waesvjvahsvalvmlovaorfvwar aszjnm caerhsa zhgjvaHsvafrooztvjiaevjvazaxomivap rhaxmnng rhcaymg waycahjzwrhrm iaz wazaiszjvwasrihmjcahszhaihjvhxsvwayzxpahsjmgtsahsvatv vjzhrm iaVzxsawzcar ahsvafrooztvayvtz aerhsahsvaurjihaortshamuawze aziahsvaig 'iajzcialrvjxvwahsjmgtsahsvaxz mlcamuahjvviaz wayzhsvwahsvafzoovcar azatmowv atomeaHsvafrooztvjiaemgowajrivaujmnahsvrjaiognyvjaz waivhazymghahsvrjawzrocahzipiaerhsazaiv ivamualgjlmivaz wawvhvjnr zhrm aHsvauzjnvjiaemgowasvzwahmahsvrjaurvowiahmahv wahmahsvaxjmliahszhaljmfrwvwaigihv z xvaumjahsvaxmnng rhcaesrovahsvaisvlsvjwiatgrwvwahsvrjauomxpiahmahsvaogisalzihgjviahszhaor vwahsvasrooirwviaHsvayozxpinrhs'iaumjtvaemgowaxmnvahmaoruvaerhsahsvajr tr tamuasznnvjam az froaziasvaiszlvwanvhzoar hmahmmoiaz warnlovnv hiaviiv hrzoaumjahsvafrooztvji'aemjpaHsvayzpvjaemgowajrivavfv avzjorvjahsz ahsvamhsvjiahmaljvlzjvahsvawmgtsaumjahsvayjvzwaz walzihjrviahszhauroovwahsvazrjaerhsahsvrjahz hzorbr tazjmnzaYcahsvahrnvahsvaig aszwaugoocajriv ahsvafrooztvaikgzjvaemgowayvazaygihor tasgyamuazxhrfrhcazialvmlovatzhsvjvwahmahjzwvatmmwiavdxsz tva veiaz waiszjvar ahsvaxznzjzwvjrvahszhawvur vwahsvrjaezcamuaoruvaHsvafrooztvaeziaigjjmg wvwaycazalzhxsemjpamuaurvowiaz wanvzwmeiavzxsam vazahvihznv hahmahsvaszjwaemjpaz wawvwrxzhrm amuahsvauzjnvjiaesmahv wvwahsvnaHsvivaurvowiaevjvahsvaoruvyommwamuahsvafrooztvaljmfrwr tahsvatjzr iafvtvhzyoviaz waujgrhiahszhaigihzr vwahsvafrooztvjiahsjmgtsmghahsvacvzjaR ahsvailjr tahsvaurvowiaemgowaxmnvazorfvaerhsahsvafryjz hatjvv amua veatjmehsaziaivvwiaime ar ahsvauvjhrovaimroayvtz ahmailjmghaz wauomgjrisaHsvauzjnvjiaemjpvwahrjvoviiocahma gjhgjvahsvivacmg taloz hiahsvrjavuumjhiav igjr tazaymg hrugoaszjfvihar ahsvanm hsiahmaxmnvaZiaignnvjazjjrfvwahsvaurvowiaemgowayvahjz iumjnvwar hmazaivzamuatmowaziahsvaxjmliajrlv vwag wvjahsvaezjnaig aHsvafrooztvjiaemgowaxmnvahmtvhsvjahmajvzlahsvaszjfvihaemjpr tairwvaycairwvahmatzhsvjahsvaujgrhiamuahsvrjaozymjaHsvazrjaemgowayvauroovwaerhsahsvaimg wiamuaozgtshvjaz waim taziahsvafrooztvjiaxvovyjzhvwahsvaxgonr zhrm amuahsvrjaszjwaemjpaHsvaszjfvihauvihrfzoaeziam vamuahsvanmihavztvjocaz hrxrlzhvwavfv hiamuahsvacvzjazahrnvamuaqmcaz wahsz pitrfr tanzjpvwaycauvzihr tawz xr taz wangirxaHsvafrooztvaikgzjvaemgowayvazwmj vwaerhsaxmomjugoayz  vjiaz watzjoz wiaz wahzyoviaemgowayvaozwv aerhsaz azyg wz xvamuaummwaz wawjr paHsvafrooztvjiaemgowawz xvaozhvar hmahsva rtshahsvrjailrjrhiaoruhvwaycahsvaiv ivamuaxmnng rhcaz wahsvaizhriuzxhrm amuazaqmyaevooawm vaZiahsvawzciatjveaismjhvjaz wahsvazrjahgj vwaxjrilaerhsahsvazjjrfzoamuazghgn ahsvafrooztvjiaemgowaljvlzjvaumjahsvaxmowvjanm hsiazsvzwaHsvcaemgowatzhsvjaemmwaumjahsvrjaurjvialjvivjfvaummwaumjahsvaer hvjaz wav igjvahszhahsvrjasmnviaevjvajvzwcahmaerhsihz wahsvaxsrooamuaer hvjaHsvaurvowiaemgowayvalomevwaz waljvlzjvwaumjahsva vdhaloz hr taivzim ahsvaimroahgj vwaz wav jrxsvwaerhsahsvajvn z hiamuahsvaszjfvihaHsvafrooztvaemgowahzpvam azakgrvhvjanmjvajvuovxhrfvanmmwaziahsvafrooztvjiahgj vwar ezjwaljvlzjr taumjahsvaom taer hvjanm hsiaEr hvjar ahsvafrooztvaeziazahrnvamuajvihaz wajv vezoazaxsz xvaumjahsvafrooztvjiahmajvxszjtvahsvrjailrjrhiaz wajvxm  vxhaerhsahsvrjauznrorviaHsvaurvowiaozcawmjnz hag wvjazayoz pvhamuai meahsvahjvviayzjvaz wairov har ahsvaxmowaHsvafrooztvjiaemgowatzhsvjazjmg wahsvrjasvzjhsiaiszjr taihmjrviaz waim tiaziahsvcaezrhvwaumjahsvajvhgj amuailjr taHsvaer hvjaimoihrxvaeziazalzjhrxgozjocailvxrzoahrnvanzjpvwaycahsvavdxsz tvamuasz wnzwvatruhiaz wahsvaxvovyjzhrm amuahsvahgj r tamuahsvacvzjaRhaeziazahrnvamuasmlvaz wajv vezoazajvnr wvjahszhavfv ar ahsvawzjpvihawzciahsvaortshaemgowajvhgj az waoruvaemgowayvtr az veaHsjmgtsmghahsvacvzjahsvafrooztvaeziatgrwvwaycazaivhamuahjzwrhrm iaz waxgihmniahszhaszwayvv alziivwawme ahsjmgtsahsvatv vjzhrm iaHsvivahjzwrhrm iaevjvahsvasvzjhamuahsvaxmnng rhcaljmfrwr tazaiv ivamuaxm hr grhcaz waxm  vxhrm ahmahsvalzihaM vaigxsahjzwrhrm aeziahsvaihmjchvoor ta rtshazahrnvaesv afrooztvjiaemgowatzhsvjazjmg wazaozjtvaxmnng zoaurjvahmaiszjvahzoviamuasvjmrinazwfv hgjvaz waeriwmnaHsvivaihmjrviaevjvanmjvahsz aqgihav hvjhzr nv hahsvcaevjvazaezcamualjvivjfr tahsvasrihmjcaz wafzogviamuahsvaxmnng rhcaz wamuarnlzjhr tarnlmjhz haoviim iahmahsvacmg tvjatv vjzhrm aZ mhsvjaxsvjrisvwahjzwrhrm aeziahsvaxjzuhr tamuatruhiawgjr tahsvaer hvjanm hsiaZiahsvawzciatjveaismjhvjaz wahsva rtshiaxmowvjafrooztvjiaemgowatzhsvjar ahsvrjasmnviahmaxjvzhvayvzghrugoaz wag rkgvarhvniahszhaemgowayvavdxsz tvwawgjr tahsvaer hvjaimoihrxvaxvovyjzhrm aHsriahjzwrhrm aeziazaezcamuavdljviir taomfvaz wazlljvxrzhrm az wamuaihjv thsv r tahsvaym wiayvhevv auznrocaz waujrv wiaHsvafrooztvjiaevjvazawrfvjivatjmglavzxsaerhsahsvrjame ag rkgvaiprooiaz wahzov hiahszhahsvcayjmgtshahmahsvaxmoovxhrfvavuumjhaHsvjvaeziahsvayozxpinrhsaesmahmrovwaycahsvaumjtvaiszlr tanvhzoar hmahmmoiaz warnlovnv hiahszhaevjvaviiv hrzoaumjawzrocaoruvaHsvayzpvjajmivayvumjvawze ahmaljvlzjvahsvayjvzwaz walzihjrviahszhauroovwahsvazrjaerhsahsvrjav hrxr tazjmnzaHsvaevzfvjailg ahsjvzwiar hmaxomhsaxjvzhr tatzjnv hiaz wayoz pvhiahszhaljmfrwvwaezjnhsaz waxmnumjhawgjr tahsvaxmowaer hvjanm hsiaHsvauzjnvjiaesmaozymjvwar ahsvaurvowialoz hr tahv wr taz waszjfvihr tahsvaxjmliahszhaigihzr vwahsvafrooztvaevjvanv az waemnv aesmap veahsvaoz war hrnzhvocag wvjihz wr tarhiajschsniaz waxcxoviaz waemjpr tar aszjnm caerhsa zhgjvahmaxmzdaumjhsarhiaymg hcaHsvaisvlsvjwiahv wvwahmahsvrjauomxpiatgrwr tahsvnahmaogisalzihgjviaz waljmhvxhr tahsvnaujmnaljvwzhmjiaHsvrjaiprooaz wafrtroz xvav igjvwahszhahsvafrooztvaszwazaihvzwcaigllocamuaemmoaz wanvzhaHsvafrooztvasvzovjazaerivaz wap meovwtvzyovaemnz ahv wvwahmahsvairxpaz war qgjvwagir tasvjap meovwtvamuasvjyiaz wajvnvwrviahmaxgjvazronv hiaz wavzivalzr aIsvaeziazaurtgjvamuajvilvxhaz wajvfvjv xvasvjaljviv xvaljmfrwr taxmnumjhaz wajvziigjz xvahmahsmivar a vvwaVzxsanvnyvjamuahsvaxmnng rhcalozcvwazafrhzoajmovaxm hjryghr tahsvrjavuumjhiaz wavdlvjhrivahmahsvaxmoovxhrfvatmmwaHsvafrooztvjiag wvjihmmwahszhahsvrjaihjv thsaozcar ahsvrjag rhcaz waxmmlvjzhrm az wahsvcaemjpvwahmtvhsvjahmamfvjxmnvahsvaxszoov tviahszhaoruvaljviv hvwaWvilrhvahsvaszjwisrliahsvcauzxvwahsvafrooztvjiajvnzr vwaihvzwuzihar ahsvrjaxmnnrhnv hahmavzxsamhsvjaz wahmahsvaoz wahszhaigihzr vwahsvnaHsvcag wvjihmmwahszhahsvrjaigxxviiaezia mhanvzigjvwaycaevzohsamjanzhvjrzoalmiiviirm iayghaycahsvaihjv thsamuahsvrjaxmnng rhcaz wahsvakgzorhcamuahsvrjajvozhrm isrliaerhsavzxsamhsvjaz waerhsahsva zhgjzoaemjowazjmg wahsvnaHsriafrooztvahsmgtsajvnmhvaz warimozhvwaeziazaisr r tavdznlovamuaeszhaxmgowayvazxsrvfvwaesv alvmlovaemjpvwahmtvhsvjar aszjnm caerhsazaiszjvwaiv ivamualgjlmivaz wazawvvlajvilvxhaumjavzxsamhsvjaz wahsvaoz wahsvcaxzoovwasmnvaHsvaivzim iaxm hr gvwahmahgj az wahsvafrooztvahsjrfvwarhialvmlovaorfr tar aszjnm caerhsahsvaoz waz waerhsavzxsamhsvjaHsvcaxvovyjzhvwahsvanrovihm viamuaoruvahmtvhsvjaujmnayrjhsiaz waevwwr tiahmahsvalziir tamuaomfvwam viavzxsavfv hazahvihznv hahmahsvav wgjr taihjv thsamuahsvrjaxmnng rhcaHsvafrooztvaeziazalozxvamuaozgtshvjaz waim tamuaszjwaemjpaz waxvovyjzhrm amuaomfvaz waomiiaHsjmgtsarhazooahsvafrooztvjiajvnzr vwaymg waycahsvahjzwrhrm iaz wafzogviahszhaszwayvv alziivwawme ahsjmgtsahsvatv vjzhrm iahsvrjaorfviazahvihznv hahmahsvalmevjamuaxmnng rhcaz wahsvaihjv thsamuahsvasgnz ailrjrhaZiahsvacvzjialziivwahsvafrooztvaxm hr gvwahmahsjrfvazahvihznv hahmahsvajvirorv xvaz wawvhvjnr zhrm amuarhialvmlovaWvilrhvahsvaxszoov tviaz waszjwisrliahsvcauzxvwahsvafrooztvjiajvnzr vwaihvzwuzihar ahsvrjaxmnnrhnv hahmavzxsamhsvjaz wahmahsvaoz wahszhaigihzr vwahsvnaHsvcag wvjihmmwahszhahsvrjaigxxviiaezia mhanvzigjvwaycaevzohsamjanzhvjrzoalmiiviirm iayghaycahsvaihjv thsamuahsvrjaxmnng rhcaz wahsvakgzorhcamuahsvrjajvozhrm isrliaerhsavzxsamhsvjaz waerhsahsva zhgjzoaemjowazjmg wahsvnaHsriafrooztvahsmgtsajvnmhvaz warimozhvwaeziazaisr r tavdznlovamuaeszhaxmgowayvazxsrvfvwaesv alvmlovaemjpvwahmtvhsvjar aszjnm caerhsazaiszjvwaiv ivamualgjlmivaz wazawvvlajvilvxhaumjavzxsamhsvjaz wahsvaoz wahsvcaxzoovwasmnv"
#NYT
nyt = "M aMxhmyvjahsva veaCmjpaHrnvialgyorisviazawvhzrovwar fvihrtzhrm ar hmazoovtzhrm iamuaivdgzoaszjziinv haztzr ihauronaljmwgxvjaSzjfvcaEvr ihvr aHsvaymnyisvooajvlmjhaovwahmaEvr ihvr iavfv hgzoazjjvihaz waxm frxhrm am axszjtviamuajzlvaz wamhsvjaivdgzoanrixm wgxhaRhasziair xvayvxmnvajvxmt rbvwaziam vamuahsvawvur r tavzjocanmnv hiamuahsvaNvHmmanmfvnv haErhsr axvjhzr axrjxoviaEvr ihvr ialjvwzhmjcayvszfrmjaszwayvv aerwvocap me aumjaimnvahrnvaNgohrlovajvlmjhvjiaszwayvv aixjznyor tahmayjvzpahsvaihmjcajvlmjhvjaJm z aUzjjmeaszwayjmgtshahsvaihmjcahma YXa veiayghaeziahgj vwazezcavfv hgzoocalgyorisr tasriavdlmivar aHsva veaCmjpvjaurfvawzciazuhvjahsvaHrnviajz arhiar fvihrtzhrm aHsvaorihamuaEvr ihvr iazxxgivjiavfv hgzoocatjveahmaivfvjzoawmbv aemnv ajz tr taujmnaevooap me azxhjviiviaorpvaZisovcaQgwwaz waTec vhsaLzohjmeahmaemnv aesmaszwaemjpvwaumjaziaorhhovaziam vawzcazhaNrjznzdamjahsvaEvr ihvr aXmnlz caEvr ihvr aeziarnnvwrzhvocaurjvwaujmnasriaxmnlz caz wavdlvoovwaujmnahsvaNmhrm aLrxhgjvaZxzwvncamuaZjhiaz waIxrv xviaR aUvyjgzjcasvaeziaumg watgrohcamuauvom caziizgohaz waiv hv xvwahmacvzjiar aljrim aHsvazoovtzhrm iaivhamuuazaxszr ajvzxhrm azianmjvaemnv axznvaumjezjwaerhsaivdgzoanrixm wgxhazoovtzhrm iaztzr iharnlmjhz haurtgjviahsjmgtsmghahsvav hvjhzr nv har wgihjcahsvauzisrm ar wgihjcalmorhrxiahsvailmjhiaemjowaz wavoivesvjvaR azwwrhrm ahmahsvalgyorxaisznr tamuaza gnyvjamuasrtsalmevjvwanvwrzaurtgjviaz waxvovyjrhrviahsvaNvHmmaNmfvnv haziarhaxznvahmayvap me aumjxvwanz caZnvjrxz iahmavdznr vahsvalmevjawc znrxiaz war ihrhghrm zoaivdrinahszhazoomevwalmevjugoanv ar afrjhgzoocavfvjcaurvowahmaxmnnrhaz waxmfvjaglazooanz  vjamuaivdgzoanrixm wgxhaImnvamuahsvanv aesmauzxvwaNvHmmazoovtzhrm iaszfvavrhsvjazlmomtrbvwaz wazhhvnlhvwahmanmfvam aerhsahsvrjaxzjvvjiamjairnlocawv rvwahsvaxszjtviaz wapvlhahsvrjalmirhrm iamualmevja mhzyocamfvjaemnv azxxgivwahsv aLjvirwv haWm zowaHjgnlamuanrixm wgxhajz tr taujmnafvjyzoaszjziinv hahmajzlvaNvz esrovazia veazoovtzhrm iaxm hr gvahmazjrivar aemjplozxviazxjmiiahsvaemjowarhiaxovzjahszhahsvaEvr ihvr aixz wzoaeziazapvcaihvlar ahsvaihjgttovahmavdlmivaz waihznlamghaivdgzoanrixm wgxh"
# instead of markov chain, treat as a single distribution
def build_freq_list(input_file):
    """
    Builds a list that represents the distribution of letters in input_file.
    """
    # create an array of 1's. 
    counts = np.ones([27])
    with open(input_file, 'r', encoding='utf8') as f:
        for _ in range(3000000):
            line = f.readline()
            if len(line) > 2:
                for i in range(len(line) - 1):
                    # if not an alphabetic character, then treat the charater as (space)
                    # ASCII capital letters start at 65.
                    char = ord(line[i].upper()) - 65 if line[i].isalpha() else 26
                    # <= 26 means ignore the characters that gives lower case letters because they could pass 
                    # the isalpha checkpoint.
                    if char <= 26:
                        counts[char] += 1
        # tranposition is for dimension match
        freq_list = (counts / np.sum(counts))
    return freq_list

# Compute empirical distribution
def text_freq_list(encrypted_message):
    # create an array of 1's. 
    counts = np.ones([27])
    for i in range(len(encrypted_message)):
        # if not an alphabetic character, then treat the charater as (space)
        # ASCII capital letters start at 65.
        char = ord(encrypted_message[i].upper()) - 65 if encrypted_message[i].isalpha() else 26
        # <= 26 means ignore the characters that gives lower case letters because they could pass 
        # the isalpha checkpoint.
        if char <= 26:
            counts[char] += 1
    # tranposition is for dimension match
    freq_list = (counts / np.sum(counts))
    return freq_list
    
# # real distribution
# array1 = build_freq_list(input_file)
# # empirical distribution
# array2 = text_freq_list(nyt)

# from scipy.optimize import linear_sum_assignment

# # Step 1: Create the cost matrix
# cost_matrix = np.abs(array1[:, np.newaxis] - array2)

# # Step 2: Apply the Hungarian algorithm
# row_ind, col_ind = linear_sum_assignment(cost_matrix)
# print(decrypt(nyt, col_ind).upper())

import string
from collections import Counter

# English letter frequency in percentage
ENGLISH_LETTER_FREQ = {
    'E': 12.02, 'T': 9.10, 'A': 8.12, 'O': 7.68, 'I': 7.31, 'N': 6.95, 'S': 6.28, 'R': 6.02,
    'H': 5.92, 'D': 4.32, 'L': 3.98, 'U': 2.88, 'C': 2.71, 'M': 2.61, 'F': 2.30, 'Y': 2.11,
    'W': 2.09, 'G': 2.03, 'P': 1.82, 'B': 1.49, 'V': 1.11, 'K': 0.69, 'X': 0.17, 'Q': 0.11,
    'J': 0.10, 'Z': 0.07
}

def frequency_analysis(ciphertext):
    # Remove spaces and non-alphabetic characters from the ciphertext
    filtered_text = ''.join(filter(str.isalpha, ciphertext.upper()))
    
    # Calculate the frequency of each letter in the ciphertext
    letter_counts = Counter(filtered_text)
    
    # Convert counts to frequencies
    total_letters = sum(letter_counts.values())
    letter_freqs = {letter: (count / total_letters) * 100 for letter, count in letter_counts.items()}
    
    # Sort letter frequencies from the ciphertext and English letter frequencies by frequency
    sorted_cipher_freq = sorted(letter_freqs.items(), key=lambda item: item[1], reverse=True)
    sorted_english_freq = sorted(ENGLISH_LETTER_FREQ.items(), key=lambda item: item[1], reverse=True)
    
    # Create a tentative substitution mapping based on frequency analysis
    tentative_mapping = {}
    for i in range(min(len(sorted_cipher_freq), len(sorted_english_freq))):
        tentative_mapping[sorted_cipher_freq[i][0]] = sorted_english_freq[i][0]
    
    return tentative_mapping, sorted_cipher_freq, sorted_english_freq

def apply_substitution(ciphertext, mapping):
    # Apply the tentative substitution mapping to the ciphertext
    substituted_text = ''.join(mapping.get(char, char) for char in ciphertext.upper())
    return substituted_text

# Example longer ciphertext (at least 200 characters)
ciphertext = """YMJ QNRJY KTHRJSY FXYNSL TK NX LZWJ HNUMJW NS JMTB YMJ QNRJY NX KFWI JSYX QJYY YMJ QNRJY KTWRFYX FYNSL NX TK HNUMJW YMJ QNRJY KTWRFYX NY KFWI QNRJY KTHRJSY KTWRFYX FXYNSL NX JMTB YMJ QNRJY KTWRFYX FYNSL TK NX HNUMJW"""

# Perform frequency analysis
mapping, cipher_freq, english_freq = frequency_analysis(ciphertext)

# Apply the substitution
decoded_text = apply_substitution(ciphertext, mapping)

print("Ciphertext:", ciphertext)
print("Decoded Text:", decoded_text)
print("Tentative Mapping:", mapping)

import string
from collections import Counter

# English letter frequency in percentage
ENGLISH_LETTER_FREQ = {
    'E': 12.02, 'T': 9.10, 'A': 8.12, 'O': 7.68, 'I': 7.31, 'N': 6.95, 'S': 6.28, 'R': 6.02,
    'H': 5.92, 'D': 4.32, 'L': 3.98, 'U': 2.88, 'C': 2.71, 'M': 2.61, 'F': 2.30, 'Y': 2.11,
    'W': 2.09, 'G': 2.03, 'P': 1.82, 'B': 1.49, 'V': 1.11, 'K': 0.69, 'X': 0.17, 'Q': 0.11,
    'J': 0.10, 'Z': 0.07
}

# Common English bigrams
COMMON_BIGRAMS = ["TH", "HE", "IN", "ER", "AN", "RE", "ND", "AT", "ON", "NT", "HA", "ES", "ST", "EN", "ED", "TO", "IT", "OU", "EA", "HI"]

# Common English trigrams
COMMON_TRIGRAMS = ["THE", "AND", "ING", "HER", "ERE", "ENT", "THA", "NTH", "WAS", "ETH", "FOR", "DTH", "HES", "HIS", "TIO", "ORD", "SHE", "NOT", "RES", "EDD"]

def frequency_analysis(ciphertext):
    # Remove spaces and non-alphabetic characters from the ciphertext
    filtered_text = ''.join(filter(str.isalpha, ciphertext.upper()))
    
    # Calculate the frequency of each letter in the ciphertext
    letter_counts = Counter(filtered_text)
    
    # Convert counts to frequencies
    total_letters = sum(letter_counts.values())
    letter_freqs = {letter: (count / total_letters) * 100 for letter, count in letter_counts.items()}
    
    # Sort letter frequencies from the ciphertext and English letter frequencies by frequency
    sorted_cipher_freq = sorted(letter_freqs.items(), key=lambda item: item[1], reverse=True)
    sorted_english_freq = sorted(ENGLISH_LETTER_FREQ.items(), key=lambda item: item[1], reverse=True)
    
    # Create a tentative substitution mapping based on frequency analysis
    tentative_mapping = {}
    for i in range(min(len(sorted_cipher_freq), len(sorted_english_freq))):
        tentative_mapping[sorted_cipher_freq[i][0]] = sorted_english_freq[i][0]
    
    return tentative_mapping, sorted_cipher_freq, sorted_english_freq

def apply_substitution(ciphertext, mapping):
    # Apply the tentative substitution mapping to the ciphertext
    substituted_text = ''.join(mapping.get(char, char) for char in ciphertext.upper())
    return substituted_text

# Further refining the mapping by checking common bigrams and trigrams
def refine_mapping(tentative_mapping, ciphertext):
    # Reverse the mapping for easy lookup
    reversed_mapping = {v: k for k, v in tentative_mapping.items()}
    
    for bigram in COMMON_BIGRAMS:
        cipher_bigram = ''.join(reversed_mapping.get(char, '?') for char in bigram)
        if '?' not in cipher_bigram and cipher_bigram in ciphertext:
            tentative_mapping[cipher_bigram[0]] = bigram[0]
            tentative_mapping[cipher_bigram[1]] = bigram[1]
    
    for trigram in COMMON_TRIGRAMS:
        cipher_trigram = ''.join(reversed_mapping.get(char, '?') for char in trigram)
        if '?' not in cipher_trigram and cipher_trigram in ciphertext:
            tentative_mapping[cipher_trigram[0]] = trigram[0]
            tentative_mapping[cipher_trigram[1]] = trigram[1]
            tentative_mapping[cipher_trigram[2]] = trigram[2]
    
    return tentative_mapping

# Example longer ciphertext (at least 200 characters)
ciphertext = """YMJ QNRJY KTHRJSY FXYNSL TK NX LZWJ HNUMJW NS JMTB YMJ QNRJY NX KFWI JSYX QJYY YMJ QNRJY KTWRFYX FYNSL NX TK HNUMJW YMJ QNRJY KTWRFYX NY KFWI QNRJY KTHRJSY KTWRFYX FXYNSL NX JMTB YMJ QNRJY KTWRFYX FYNSL TK NX HNUMJW"""

# Perform frequency analysis
mapping, cipher_freq, english_freq = frequency_analysis(ciphertext)

# Refine the mapping using common bigrams and trigrams
mapping = refine_mapping(mapping, ciphertext)

# Apply the substitution
decoded_text = apply_substitution(ciphertext, mapping)

print("Ciphertext:", ciphertext)
print("Decoded Text:", decoded_text)
print("Tentative Mapping:", mapping)


Ciphertext: YMJ QNRJY KTHRJSY FXYNSL TK NX LZWJ HNUMJW NS JMTB YMJ QNRJY NX KFWI JSYX QJYY YMJ QNRJY KTWRFYX FYNSL NX TK HNUMJW YMJ QNRJY KTWRFYX NY KFWI QNRJY KTHRJSY KTWRFYX FXYNSL NX JMTB YMJ QNRJY KTWRFYX FYNSL TK NX HNUMJW
Decoded Text: ERT UAOTE NSCOTLE HIEALM SN AI MGDT CAFRTD AL TRSY ERT UAOTE AI NHDW TLEI UTEE ERT UAOTE NSDOHEI HEALM AI SN CAFRTD ERT UAOTE NSDOHEI AE NHDW UAOTE NSCOTLE NSDOHEI HIEALM AI TRSY ERT UAOTE NSDOHEI HEALM SN AI CAFRTD
Tentative Mapping: {'Y': 'E', 'J': 'T', 'N': 'A', 'R': 'O', 'X': 'I', 'K': 'N', 'T': 'S', 'M': 'R', 'F': 'H', 'W': 'D', 'S': 'L', 'Q': 'U', 'H': 'C', 'L': 'M', 'U': 'F', 'B': 'Y', 'I': 'W', 'Z': 'G'}
Ciphertext: YMJ QNRJY KTHRJSY FXYNSL TK NX LZWJ HNUMJW NS JMTB YMJ QNRJY NX KFWI JSYX QJYY YMJ QNRJY KTWRFYX FYNSL NX TK HNUMJW YMJ QNRJY KTWRFYX NY KFWI QNRJY KTHRJSY KTWRFYX FXYNSL NX JMTB YMJ QNRJY KTWRFYX FYNSL TK NX HNUMJW
Decoded Text: ERT UAOTE NSCOTLE HIEALM SN AI MGDT CAFRTD AL TRSY ERT UAOTE AI NHDW TLEI UTEE ERT UAOTE NSDOHEI HEA